In [20]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [21]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [22]:
subject = 'Alagoas - Consumo de Cimento (t)'
split_index = 203 #Referente aos 230 anos de input  
train_split = split_index + 1 - 12*3

In [23]:
data = pd.read_csv('2003_mo_model_input_AL.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,Alagoas - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Alagoas - value,Alagoas - Desemprego,Alagoas - Produção de Cimento (t),Alagoas - PIB - Estadual,Alagoas - PIB - Construção Civil,Alagoas - PIB - Per Capita,Alagoas - PIB - Preços de Mercado,Alagoas - Consumo de Cimento (t)
0,2003-1,0.635160,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,0.294434,8.765721,30.779440,2.590528e+07,1.303625e+06,7.576443,2.359888e+07,30.109
1,2003-2,0.635571,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,0.296211,8.760196,30.599132,2.593685e+07,1.305020e+06,7.578654,2.360861e+07,27.684
2,2003-3,0.635982,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,0.297521,8.754671,30.545910,2.596843e+07,1.306414e+06,7.580866,2.361833e+07,26.534
3,2003-4,0.636393,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,0.298766,8.749145,30.377624,2.600000e+07,1.307809e+06,7.583078,2.362806e+07,25.109
4,2003-5,0.636804,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,0.299945,8.743620,30.464900,2.603157e+07,1.309203e+06,7.585289,2.363779e+07,25.171
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.587645,NaN,24.556360,NaN,NaN,NaN,NaN,46.546
236,2022-9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.587084,NaN,24.501314,NaN,NaN,NaN,NaN,47.781
237,2022-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.585891,NaN,24.609100,NaN,NaN,NaN,NaN,51.171
238,2022-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.584366,NaN,24.538488,NaN,NaN,NaN,NaN,49.126


In [24]:
input_data = data.iloc[:split_index + 1,1:-1]
# input_data = data.iloc[:split_index + 1,:]
input_data = (input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)
input_data

,Alagoas - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Alagoas - value,Alagoas - Desemprego,Alagoas - Produção de Cimento (t),Alagoas - PIB - Estadual,Alagoas - PIB - Construção Civil,Alagoas - PIB - Per Capita,Alagoas - PIB - Preços de Mercado
0,-2.146764,2.782450,4.506880,2.067266,2.574314,-2.064648,-2.469876,3.184489,-1.449375,-0.937577,-0.789529,-1.703997,-1.236973,-1.975005,-1.922747
1,-2.113412,2.407943,4.328460,1.285816,2.334870,-2.037913,-2.431875,3.029073,-1.423792,-0.939615,-0.803493,-1.687205,-1.181428,-1.949747,-1.897135
2,-2.080060,2.179073,4.129086,2.919965,2.221334,-2.011179,-2.393874,2.926505,-1.404924,-0.941652,-0.807615,-1.670414,-1.125883,-1.924488,-1.871523
3,-2.046707,2.077086,3.911409,1.142823,2.040542,-1.984445,-2.355872,2.828220,-1.386985,-0.943690,-0.820649,-1.653622,-1.070338,-1.899230,-1.845911
4,-2.013355,1.942128,3.663912,2.355956,1.912744,-1.957710,-2.317871,3.036493,-1.370013,-0.945728,-0.813889,-1.636831,-1.014793,-1.873971,-1.820299
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,1.183322,-0.689886,-0.724085,-1.189161,3.148408,1.378950,0.117681,0.370628,1.308074,0.919145,-1.198723,0.963258,-0.926262,0.929504,0.848917
200,1.177023,-0.441954,-0.736434,-1.296499,3.219670,1.391539,0.092456,0.427775,1.359203,0.913671,-1.181757,0.946383,-0.898324,0.909729,0.830432
201,1.170724,-0.132782,-0.738433,-1.444029,3.421082,1.404128,0.067231,0.538287,1.405650,0.908197,-1.171317,0.929508,-0.870386,0.889955,0.811947
202,1.164425,0.084061,-0.738236,-1.444143,3.373840,1.416717,0.042006,0.666156,1.459075,0.902724,-1.180748,0.912633,-0.842448,0.870180,0.793463


In [25]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0      27.684
1      26.534
2      25.109
3      25.171
4      22.645
        ...  
235    47.781
236    51.171
237    49.126
238    49.126
239       NaN
Name: Alagoas - Consumo de Cimento (t), Length: 240, dtype: float64

In [27]:
# input para treinamento
train_input = input_data.iloc[:train_split]
train_input

,Alagoas - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Alagoas - value,Alagoas - Desemprego,Alagoas - Produção de Cimento (t),Alagoas - PIB - Estadual,Alagoas - PIB - Construção Civil,Alagoas - PIB - Per Capita,Alagoas - PIB - Preços de Mercado
0,-2.146764,2.782450,4.506880,2.067266,2.574314,-2.064648,-2.469876,3.184489,-1.449375,-0.937577,-0.789529,-1.703997,-1.236973,-1.975005,-1.922747
1,-2.113412,2.407943,4.328460,1.285816,2.334870,-2.037913,-2.431875,3.029073,-1.423792,-0.939615,-0.803493,-1.687205,-1.181428,-1.949747,-1.897135
2,-2.080060,2.179073,4.129086,2.919965,2.221334,-2.011179,-2.393874,2.926505,-1.404924,-0.941652,-0.807615,-1.670414,-1.125883,-1.924488,-1.871523
3,-2.046707,2.077086,3.911409,1.142823,2.040542,-1.984445,-2.355872,2.828220,-1.386985,-0.943690,-0.820649,-1.653622,-1.070338,-1.899230,-1.845911
4,-2.013355,1.942128,3.663912,2.355956,1.912744,-1.957710,-2.317871,3.036493,-1.370013,-0.945728,-0.813889,-1.636831,-1.014793,-1.873971,-1.820299
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,1.324514,-0.742853,-0.619453,0.617009,-0.970725,0.980624,0.764972,-1.322806,1.121339,1.186919,-1.069189,1.177341,-1.214407,1.097518,1.109211
164,1.323316,-0.787367,-0.611176,0.459632,-0.841670,0.991608,0.753473,-1.355523,1.103076,1.173733,-1.161695,1.181165,-1.230415,1.105489,1.111823
165,1.322118,-0.757940,-0.599627,0.493659,-0.838453,1.002592,0.741973,-1.376175,1.083984,1.160546,-1.230868,1.184990,-1.246424,1.113460,1.114436
166,1.320921,-0.917469,-0.589237,0.617024,-1.006001,1.013577,0.730473,-1.414146,1.070055,1.147360,-1.298996,1.188814,-1.262433,1.121431,1.117048


In [28]:
# Alvo para treinamento
train_target = target_data.iloc[:train_split]
train_target

0      27.684000
1      26.534000
2      25.109000
3      25.171000
4      22.645000
         ...    
163    42.491371
164    41.535029
165    40.578686
166    39.622343
167    38.666000
Name: Alagoas - Consumo de Cimento (t), Length: 168, dtype: float64

In [29]:
#input de test (Ano 2021)
test_input = input_data.iloc[train_split:]
test_input

,Alagoas - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Alagoas - value,Alagoas - Desemprego,Alagoas - Produção de Cimento (t),Alagoas - PIB - Estadual,Alagoas - PIB - Construção Civil,Alagoas - PIB - Per Capita,Alagoas - PIB - Preços de Mercado
168,1.318526,-1.032458,-0.585256,-0.505289,-1.202612,1.035545,0.707473,-1.446117,1.040654,1.120987,-1.422705,1.196462,-1.294450,1.137372,1.122273
169,1.317834,-1.077854,-0.581056,-0.373612,-1.139987,1.046449,0.692399,-1.526274,1.028253,1.113532,-1.481842,1.193806,-1.294397,1.136752,1.115681
170,1.317142,-1.036724,-0.574769,-0.291560,-1.053534,1.057352,0.677324,-1.610341,1.015519,1.106076,-1.530176,1.191149,-1.294344,1.136131,1.109088
171,1.316450,-1.013400,-0.567021,-0.450118,-0.932974,1.068256,0.662249,-1.601301,1.002465,1.098621,-1.593615,1.188492,-1.294290,1.135511,1.102495
172,1.315758,-1.059228,-0.557842,-0.909054,-0.763756,1.079159,0.647175,-1.618284,0.986107,1.091166,-1.643819,1.185835,-1.294237,1.134890,1.095903
173,1.315066,-1.170032,-0.548012,0.387442,-0.644652,1.090063,0.632100,-1.602348,0.978396,1.083710,-1.698151,1.183179,-1.294184,1.134270,1.089310
174,1.314374,-1.115621,-0.537244,-0.937589,-0.541601,1.100966,0.617026,-1.686659,0.970719,1.076255,-1.715076,1.180522,-1.294131,1.133650,1.082718
175,1.313682,-1.182159,-0.524149,-1.212466,-0.565747,1.111870,0.601951,-1.688184,0.963104,1.068800,-1.713343,1.177865,-1.294077,1.133029,1.076125
176,1.312990,-1.270623,-0.519647,-0.499856,-0.536256,1.122773,0.586877,-1.692173,0.955578,1.061345,-1.705266,1.175209,-1.294024,1.132409,1.069532
177,1.312298,-1.321931,-0.510389,-0.120028,-0.544036,1.133677,0.571802,-1.690760,0.948164,1.053889,-1.685174,1.172552,-1.293971,1.131788,1.062940


In [30]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[train_split:split_index + 1]
test_target

168    32.321
169    41.937
170    31.982
171    28.499
172    22.626
173    21.340
174    26.848
175    28.074
176    29.941
177    28.977
178    31.580
179    29.991
180    26.820
181    29.446
182    27.666
183    22.013
184    28.025
185    25.648
186    31.782
187    27.768
188    33.073
189    30.389
190    27.657
191    32.388
192    30.378
193    30.641
194    32.337
195    36.405
196    22.499
197    27.961
198    32.378
199    32.582
200    42.703
201    38.584
202    36.503
203    43.323
Name: Alagoas - Consumo de Cimento (t), dtype: float64

In [31]:
def validation_splitter(df, div_factor):
    split_factor = len(df)//div_factor
    positions_to_drop = []
    for i in range(split_factor):
        pos = df.shape[0] - (i*6 + 1)
        positions_to_drop.append(pos)
    
    return df.drop(positions_to_drop), df.iloc[positions_to_drop]

In [32]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=100)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    train, train_val = validation_splitter(train_input, 8)
    target,target_val = validation_splitter(train_target, 8)
#     display(train)
#     display(train_val)
#     display(target)
#     display(target_val)
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val, target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [33]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(25)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][100:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [35]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[2772607606, 2333565501, 2043342657, 2221482550, 472559116, 3246216639, 3938558512, 2708359791, 513449404, 3319624788, 4162578690, 1278098209, 2059087650, 3894751891, 2182249241, 1222227574, 2047637226, 3259805526, 1407790155, 424044040, 231898124, 3531857747, 3870654282, 3846140577, 893249733]


Step: 0 ___________________________________________
val_loss: 19.628711700439453
winner_seed: 2772607606


Step: 1 ___________________________________________
val_loss: 18.02029800415039
winner_seed: 2333565501


Step: 2 ___________________________________________
val_loss: 19.01398277282715


Step: 3 ___________________________________________
val_loss: 16.199064254760742
winner_seed: 2221482550


Step: 4 ___________________________________________
val_loss: 18.474363327026367


Step: 5 ___________________________________________
val_loss: 16.40768814086914


Step: 6 ___________________________________________
val_loss: 19.316978454589844


Step: 7 ___________________________________________


In [36]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
5/5 [==============================] - 0s 25ms/step - loss: 2064.6521 - val_loss: 2557.6284
Epoch 2/10000
5/5 [==============================] - 0s 5ms/step - loss: 1744.4268 - val_loss: 1433.8445
Epoch 3/10000
5/5 [==============================] - 0s 4ms/step - loss: 1622.4667 - val_loss: 1049.1791
Epoch 4/10000
5/5 [==============================] - 0s 4ms/step - loss: 741.4020 - val_loss: 229.4003
Epoch 5/10000
5/5 [==============================] - 0s 4ms/step - loss: 135.6669 - val_loss: 106.3464
Epoch 6/10000
5/5 [==============================] - 0s 5ms/step - loss: 68.7447 - val_loss: 99.6047
Epoch 7/10000
5/5 [==============================] - 0s 4ms/step - loss: 52.3194 - val_loss: 129.5311
Epoch 8/10000
5/5 [==============================] - 0s 4ms/step - loss: 39.7440 - val_loss: 86.9604
Epoch 9/10000
5/5 [==============================] - 0s 4ms/step - loss: 46.1158 - val_loss: 33.0114
Epoch 10/10000
5/5 [==============================] - 0s 4ms/step - loss:

5/5 [==============================] - 0s 4ms/step - loss: 17.9350 - val_loss: 21.1423
Epoch 82/10000
5/5 [==============================] - 0s 4ms/step - loss: 17.8256 - val_loss: 23.5383
Epoch 83/10000
5/5 [==============================] - 0s 4ms/step - loss: 18.5778 - val_loss: 22.2080
Epoch 84/10000
5/5 [==============================] - 0s 4ms/step - loss: 17.5598 - val_loss: 29.6434
Epoch 85/10000
5/5 [==============================] - 0s 4ms/step - loss: 18.2575 - val_loss: 24.4585
Epoch 86/10000
5/5 [==============================] - 0s 4ms/step - loss: 18.6937 - val_loss: 22.7380
Epoch 87/10000
5/5 [==============================] - 0s 4ms/step - loss: 16.9910 - val_loss: 21.6142
Epoch 88/10000
5/5 [==============================] - 0s 4ms/step - loss: 18.9985 - val_loss: 23.4188
Epoch 89/10000
5/5 [==============================] - 0s 4ms/step - loss: 18.6755 - val_loss: 26.9722
Epoch 90/10000
5/5 [==============================] - 0s 4ms/step - loss: 20.7678 - val_loss: 23.

Epoch 161/10000
5/5 [==============================] - 0s 4ms/step - loss: 15.9751 - val_loss: 20.5312
Epoch 162/10000
5/5 [==============================] - 0s 4ms/step - loss: 17.2320 - val_loss: 22.3193
Epoch 163/10000
5/5 [==============================] - 0s 6ms/step - loss: 17.3042 - val_loss: 19.1826
Epoch 164/10000
5/5 [==============================] - 0s 5ms/step - loss: 17.1689 - val_loss: 22.6662
Epoch 165/10000
5/5 [==============================] - 0s 4ms/step - loss: 17.3830 - val_loss: 20.1489
Epoch 166/10000
5/5 [==============================] - 0s 4ms/step - loss: 16.0655 - val_loss: 23.6070
Epoch 167/10000
5/5 [==============================] - 0s 4ms/step - loss: 18.5939 - val_loss: 19.3382
Epoch 168/10000
5/5 [==============================] - 0s 5ms/step - loss: 17.3435 - val_loss: 20.1479
Epoch 169/10000
5/5 [==============================] - 0s 4ms/step - loss: 17.2910 - val_loss: 20.8503
Epoch 170/10000
5/5 [==============================] - 0s 4ms/step - loss

5/5 [==============================] - 0s 4ms/step - loss: 15.6385 - val_loss: 19.1428
Epoch 241/10000
5/5 [==============================] - 0s 4ms/step - loss: 17.3287 - val_loss: 20.9794
Epoch 242/10000
5/5 [==============================] - 0s 4ms/step - loss: 17.7925 - val_loss: 21.1359
Epoch 243/10000
5/5 [==============================] - 0s 4ms/step - loss: 16.3594 - val_loss: 25.7004
Epoch 244/10000
5/5 [==============================] - 0s 4ms/step - loss: 16.8885 - val_loss: 24.1228
Epoch 245/10000
5/5 [==============================] - 0s 4ms/step - loss: 17.0819 - val_loss: 23.8982
Epoch 246/10000
5/5 [==============================] - 0s 4ms/step - loss: 16.2791 - val_loss: 24.0481
Epoch 247/10000
5/5 [==============================] - 0s 4ms/step - loss: 16.2744 - val_loss: 20.8469
Epoch 248/10000
5/5 [==============================] - 0s 4ms/step - loss: 15.5727 - val_loss: 21.0286
Epoch 249/10000
5/5 [==============================] - 0s 4ms/step - loss: 16.7743 - val_

5/5 [==============================] - 0s 4ms/step - loss: 16.4098 - val_loss: 20.8571
Epoch 320/10000
5/5 [==============================] - 0s 4ms/step - loss: 15.2551 - val_loss: 20.2475
Epoch 321/10000
5/5 [==============================] - 0s 4ms/step - loss: 17.4252 - val_loss: 29.2768
Epoch 322/10000
5/5 [==============================] - 0s 4ms/step - loss: 17.4192 - val_loss: 22.9163
Epoch 323/10000
5/5 [==============================] - 0s 4ms/step - loss: 16.9533 - val_loss: 20.4724
Epoch 324/10000
5/5 [==============================] - 0s 4ms/step - loss: 15.1805 - val_loss: 21.2394
Epoch 325/10000
5/5 [==============================] - 0s 4ms/step - loss: 17.4702 - val_loss: 30.2127
Epoch 326/10000
5/5 [==============================] - 0s 4ms/step - loss: 17.9587 - val_loss: 27.5307
Epoch 327/10000
5/5 [==============================] - 0s 4ms/step - loss: 18.0438 - val_loss: 18.9670
Epoch 328/10000
5/5 [==============================] - 0s 6ms/step - loss: 15.8299 - val_

...

Epoch 1583/10000
5/5 [==============================] - 0s 4ms/step - loss: 12.1512 - val_loss: 17.8088
Epoch 1584/10000
5/5 [==============================] - 0s 4ms/step - loss: 11.9692 - val_loss: 17.3360
Epoch 1585/10000
5/5 [==============================] - 0s 4ms/step - loss: 12.3496 - val_loss: 16.9393
Epoch 1586/10000
5/5 [==============================] - 0s 4ms/step - loss: 12.0798 - val_loss: 18.7270
Epoch 1587/10000
5/5 [==============================] - 0s 4ms/step - loss: 12.6396 - val_loss: 17.9848
Epoch 1588/10000
5/5 [==============================] - 0s 4ms/step - loss: 12.0452 - val_loss: 16.5889
Epoch 1589/10000
5/5 [==============================] - 0s 4ms/step - loss: 12.1354 - val_loss: 19.5128
Epoch 1590/10000
5/5 [==============================] - 0s 4ms/step - loss: 13.3620 - val_loss: 17.9584
Epoch 1591/10000
5/5 [==============================] - 0s 4ms/step - loss: 17.0562 - val_loss: 21.7534
Epoch 1592/10000
5/5 [==============================] - 0s 4ms/s

Epoch 1662/10000
5/5 [==============================] - 0s 4ms/step - loss: 13.6738 - val_loss: 18.1011
Epoch 1663/10000
5/5 [==============================] - 0s 4ms/step - loss: 14.2873 - val_loss: 19.7183
Epoch 1664/10000
5/5 [==============================] - 0s 4ms/step - loss: 14.7234 - val_loss: 18.8006
Epoch 1665/10000
5/5 [==============================] - 0s 4ms/step - loss: 13.5622 - val_loss: 19.0216
Epoch 1666/10000
5/5 [==============================] - 0s 4ms/step - loss: 15.5023 - val_loss: 20.6300
Epoch 1667/10000
5/5 [==============================] - 0s 4ms/step - loss: 13.7155 - val_loss: 17.5272
Epoch 1668/10000
5/5 [==============================] - 0s 4ms/step - loss: 14.9798 - val_loss: 20.6340
Epoch 1669/10000
5/5 [==============================] - 0s 4ms/step - loss: 14.5368 - val_loss: 19.0466
Epoch 1670/10000
5/5 [==============================] - 0s 4ms/step - loss: 13.5443 - val_loss: 18.3526
Epoch 1671/10000
5/5 [==============================] - 0s 4ms/s

Epoch 1741/10000
5/5 [==============================] - 0s 4ms/step - loss: 13.3093 - val_loss: 19.2944
Epoch 1742/10000
5/5 [==============================] - 0s 4ms/step - loss: 13.0655 - val_loss: 20.1445
Epoch 1743/10000
5/5 [==============================] - 0s 4ms/step - loss: 14.1693 - val_loss: 22.2669
Epoch 1744/10000
5/5 [==============================] - 0s 4ms/step - loss: 12.8338 - val_loss: 19.2504
Epoch 1745/10000
5/5 [==============================] - 0s 5ms/step - loss: 13.0807 - val_loss: 19.8042
Epoch 1746/10000
5/5 [==============================] - 0s 4ms/step - loss: 13.0709 - val_loss: 20.4708
Epoch 1747/10000
5/5 [==============================] - 0s 4ms/step - loss: 12.4410 - val_loss: 19.6726
Epoch 1748/10000
5/5 [==============================] - 0s 4ms/step - loss: 13.2356 - val_loss: 21.7988
Epoch 1749/10000
5/5 [==============================] - 0s 4ms/step - loss: 13.1867 - val_loss: 20.2985
Epoch 1750/10000
5/5 [==============================] - 0s 4ms/s

Epoch 1820/10000
5/5 [==============================] - 0s 4ms/step - loss: 12.4441 - val_loss: 21.0599
Epoch 1821/10000
5/5 [==============================] - 0s 4ms/step - loss: 13.7395 - val_loss: 20.4379
Epoch 1822/10000
5/5 [==============================] - 0s 4ms/step - loss: 12.4413 - val_loss: 21.2425
Epoch 1823/10000
5/5 [==============================] - 0s 4ms/step - loss: 13.3444 - val_loss: 21.8560
Epoch 1824/10000
5/5 [==============================] - 0s 4ms/step - loss: 13.2915 - val_loss: 21.3082
Epoch 1825/10000
5/5 [==============================] - 0s 4ms/step - loss: 12.9488 - val_loss: 21.3615
Epoch 1826/10000
5/5 [==============================] - 0s 4ms/step - loss: 12.5369 - val_loss: 20.1249
Epoch 1827/10000
5/5 [==============================] - 0s 4ms/step - loss: 12.4838 - val_loss: 20.5284
Epoch 1828/10000
5/5 [==============================] - 0s 4ms/step - loss: 14.5899 - val_loss: 28.9312
Epoch 1829/10000
5/5 [==============================] - 0s 4ms/s

Epoch 1899/10000
5/5 [==============================] - 0s 4ms/step - loss: 13.4265 - val_loss: 20.5415
Epoch 1900/10000
5/5 [==============================] - 0s 4ms/step - loss: 12.7341 - val_loss: 21.9585
Epoch 1901/10000
5/5 [==============================] - 0s 5ms/step - loss: 12.8424 - val_loss: 20.7243
Epoch 1902/10000
5/5 [==============================] - 0s 4ms/step - loss: 13.4681 - val_loss: 20.4868
Epoch 1903/10000
5/5 [==============================] - 0s 4ms/step - loss: 12.4635 - val_loss: 19.2074
Epoch 1904/10000
5/5 [==============================] - 0s 4ms/step - loss: 12.2082 - val_loss: 20.3257
Epoch 1905/10000
5/5 [==============================] - 0s 4ms/step - loss: 13.8573 - val_loss: 21.8096
Epoch 1906/10000
5/5 [==============================] - 0s 4ms/step - loss: 13.9653 - val_loss: 20.5558
Epoch 1907/10000
5/5 [==============================] - 0s 4ms/step - loss: 13.0146 - val_loss: 20.8760
Epoch 1908/10000
5/5 [==============================] - 0s 7ms/s

In [37]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    for i in range(len(test_target)):
        prediction = model.predict(test_input.iloc[i:i+1])
        error = np.abs(prediction - test_target[start_target + i])
        errors.append(error)
        error_percent.append(error / test_target[start_target + i])
        print(f"Month-{i + 1} - Error: {error}")
    
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [38]:
errors, mae, mape = mae_mape_calculator(trained_model, test_input, test_target, 168)

1/1 [==============================] - 0s 51ms/step
Month-1 - Error: [[7.8380203]]
1/1 [==============================] - 0s 14ms/step
Month-2 - Error: [[1.6804543]]
1/1 [==============================] - 0s 13ms/step
Month-3 - Error: [[8.307447]]
1/1 [==============================] - 0s 13ms/step
Month-4 - Error: [[10.199242]]
1/1 [==============================] - 0s 13ms/step
Month-5 - Error: [[32.152077]]
1/1 [==============================] - 0s 13ms/step
Month-6 - Error: [[19.088215]]
1/1 [==============================] - 0s 13ms/step
Month-7 - Error: [[40.403774]]
1/1 [==============================] - 0s 13ms/step
Month-8 - Error: [[41.92343]]
1/1 [==============================] - 0s 13ms/step
Month-9 - Error: [[27.796228]]
1/1 [==============================] - 0s 13ms/step
Month-10 - Error: [[9.281652]]
1/1 [==============================] - 0s 13ms/step
Month-11 - Error: [[8.7584095]]
1/1 [==============================] - 0s 13ms/step
Month-12 - Error: [[43.839687]]
1/1 

In [39]:
display(mae)
display(mape)

38.1156

1.2585807

In [42]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum} ")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input.iloc[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [43]:
errors, mae, mape = year_mae_mape_calculator(trained_model, test_input, test_target, 168)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 14ms/step
Ano-0: |Prediction[[229.79332]] - Target[354.11599999999993]| =  Error: [[124.32268]]; MAPE:[[0.35107896]] 
1/1 [==============================] - 0s 13ms/step
Ano-0: |Prediction[[-166.81042]] - Target[342.67499999999995]| =  Error: [[509.4854]]; MAPE:[[1.486789]] 
1/1 [==============================] - 0s 13ms/step
Ano-11: |Prediction[[-205.11371]] - Target[406.294]| =  Error: [[611.4077]]


[array([[124.32268]], dtype=float32),
 array([[509.4854]], dtype=float32),
 array([[611.4077]], dtype=float32)]

415.07193

1.1142362